# HW06

請針對 Online_Retail001.xlsx 資料集，撰寫以下作業內容：

1. 選擇商品數量及購買金額 (高、中、低) 作為分群特徵，使用任一分群演算法，將其分成適當的群數。
2. 找出 RFM 分數最高的前 200 名顧客， 並將其分成 3 群。


In [1]:
import altair as alt

alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [2]:
import polars as pl

df = pl.read_excel("Online_Retail001.xlsx")

## 探索性分析


In [3]:
df.describe()

statistic,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
str,str,str,str,f64,str,f64,f64,str
"""count""","""10000""","""10000""","""9982""",10000.0,"""10000""",10000.0,7526.0,"""10000"""
"""null_count""","""0""","""0""","""18""",0.0,"""0""",0.0,2474.0,"""0"""
"""mean""",null,null,null,10.1583,"""2011-07-02 06:06:49.842000""",3.8817601,15266.94725,null
"""std""",null,null,null,57.905129,null,14.863321,1704.574949,null
"""min""","""536370""","""10120""",""" 50'S CHRISTMAS GIFT BAG LARGE""",-600.0,"""2010-12-01 08:45:00""",0.0,12347.0,"""Australia"""
"""25%""",null,null,null,1.0,"""2011-03-24 13:12:00""",1.25,13908.0,null
"""50%""",null,null,null,3.0,"""2011-07-15 14:05:00""",2.08,15128.0,null
"""75%""",null,null,null,12.0,"""2011-10-17 15:06:00""",4.13,16759.0,null
"""max""","""C580180""","""gift_0001_40""","""wrongly sold as sets""",4000.0,"""2011-12-09 12:50:00""",736.38,18283.0,"""Unspecified"""


去掉 CustomerID 為空值的資料。


In [4]:
df = df.drop_nulls("CustomerID")

In [5]:
df.describe()

statistic,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
str,str,str,str,f64,str,f64,f64,str
"""count""","""7526""","""7526""","""7526""",7526.0,"""7526""",7526.0,7526.0,"""7526"""
"""null_count""","""0""","""0""","""0""",0.0,"""0""",0.0,0.0,"""0"""
"""mean""",null,null,null,12.062184,"""2011-07-07 17:45:40.619000""",3.088628,15266.94725,null
"""std""",null,null,null,45.948289,null,7.025467,1704.574949,null
"""min""","""536370""","""10120""",""" 50'S CHRISTMAS GIFT BAG LARGE""",-300.0,"""2010-12-01 08:45:00""",0.001,12347.0,"""Australia"""
"""25%""",null,null,null,2.0,"""2011-04-03 14:24:00""",1.25,13908.0,null
"""50%""",null,null,null,5.0,"""2011-07-24 15:46:00""",1.95,15128.0,null
"""75%""",null,null,null,12.0,"""2011-10-18 13:41:00""",3.75,16759.0,null
"""max""","""C580180""","""POST""","""ZINC WILLIE WINKIE CANDLE STI…",2880.0,"""2011-12-09 12:50:00""",300.0,18283.0,"""Unspecified"""


去掉負數的 Quantity。


In [6]:
df = df.filter(pl.col("Quantity") > 0)

計算 Sales。


In [7]:
df = df.with_columns(Sales=pl.col("Quantity") * pl.col("UnitPrice"))

In [8]:
df

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
str,str,str,i64,datetime[ms],f64,i64,str,f64
"""536370""","""22726""","""ALARM CLOCK BAKELIKE GREEN""",12,2010-12-01 08:45:00,3.75,12583,"""France""",45.0
"""536375""","""71053""","""WHITE METAL LANTERN""",6,2010-12-01 09:32:00,3.39,17850,"""United Kingdom""",20.34
"""536381""","""21934""","""SKULL SHOULDER BAG""",10,2010-12-01 09:41:00,1.65,15311,"""United Kingdom""",16.5
"""536381""","""47580""","""TEA TIME DES TEA COSY""",2,2010-12-01 09:41:00,2.55,15311,"""United Kingdom""",5.1
"""536384""","""22424""","""ENAMEL BREAD BIN CREAM""",8,2010-12-01 09:53:00,10.95,18074,"""United Kingdom""",87.6
…,…,…,…,…,…,…,…,…
"""581538""","""22991""","""GIRAFFE WOODEN RULER""",1,2011-12-09 11:34:00,1.95,14446,"""United Kingdom""",1.95
"""581538""","""23084""","""RABBIT NIGHT LIGHT""",2,2011-12-09 11:34:00,2.08,14446,"""United Kingdom""",4.16
"""581582""","""23498""","""CLASSIC BICYCLE CLIPS """,12,2011-12-09 12:21:00,1.45,17581,"""United Kingdom""",17.4


## 將商品數量、購買金額進行分群


### K-Means 分群法


彙整資料，計算每個 CustomerID 的商品數量及購買金額。


In [9]:
grouped_df = df.group_by("CustomerID").agg(Quantity=pl.sum("Quantity"), Sales=pl.sum("Sales"))

In [10]:
grouped_df

CustomerID,Quantity,Sales
i64,i64,f64
16539,24,20.4
17703,24,20.04
15342,12,23.4
14300,12,10.2
16146,24,40.56
…,…,…
14002,108,8.64
16768,6,15.7
13743,36,23.4


對 Quantity 和 Sales 進行標準化。


In [11]:
from typing import cast
import pandas as pd
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.set_output(transform="pandas")

scaled_df = grouped_df.to_pandas().set_index("CustomerID")
transformed_scaled_df = cast(pd.DataFrame, scaler.fit_transform(scaled_df))

standardized_grouped_df = pl.from_pandas(transformed_scaled_df, include_index=True)

In [12]:
standardized_grouped_df

CustomerID,Quantity,Sales
i64,f64,f64
16539,-0.110585,-0.184449
17703,-0.110585,-0.185961
15342,-0.197265,-0.171851
14300,-0.197265,-0.227281
16146,-0.110585,-0.099793
…,…,…
14002,0.496172,-0.233832
16768,-0.240605,-0.204185
13743,-0.023906,-0.171851


使用 KMeans 將其分成適當的群數。


In [13]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

rfm_score_df_clustering = standardized_grouped_df.select("Quantity", "Sales")

max_silhouette_score = -1
clusterer: KMeans | None = None

for n in range(3, 9):
    kmeans = KMeans(n_clusters=n).fit(rfm_score_df_clustering)
    silhouette_avg = silhouette_score(rfm_score_df_clustering.to_numpy(), kmeans.labels_)

    print(f"n_clusters: {n}, silhouette_score: {silhouette_avg}, best: {silhouette_avg > max_silhouette_score}")

    if silhouette_avg > max_silhouette_score:
        max_silhouette_score = silhouette_avg
        clusterer = kmeans

assert clusterer is not None

grouped_df = grouped_df.with_columns(cluster=pl.Series("cluster", clusterer.labels_))

n_clusters: 3, silhouette_score: 0.965105782939181, best: True
n_clusters: 4, silhouette_score: 0.8904630625851387, best: False
n_clusters: 5, silhouette_score: 0.9275722524598748, best: False
n_clusters: 6, silhouette_score: 0.6951066784765895, best: False
n_clusters: 7, silhouette_score: 0.7663442829310423, best: False
n_clusters: 8, silhouette_score: 0.7068874628773791, best: False


In [14]:
grouped_df

CustomerID,Quantity,Sales,cluster
i64,i64,f64,i32
16539,24,20.4,0
17703,24,20.04,0
15342,12,23.4,0
14300,12,10.2,0
16146,24,40.56,0
…,…,…,…
14002,108,8.64,0
16768,6,15.7,0
13743,36,23.4,0


In [15]:
grouped_df.plot.scatter("Quantity", "Sales", color="cluster:N")

alt.Chart(...)

如上圖，可以簡單分成 3 群。


### DBSCAN 分群法


另使用 DBSCAN 方法進行分群（不過針對這個資料的效果不佳，僅供參考）。


In [16]:
from sklearn.cluster import DBSCAN

clusterer = DBSCAN(eps=0.1, min_samples=5).fit(rfm_score_df_clustering.to_numpy())

dbs_grouped_df = grouped_df.with_columns(cluster=pl.Series("cluster", clusterer.labels_))

In [17]:
dbs_grouped_df

CustomerID,Quantity,Sales,cluster
i64,i64,f64,i64
16539,24,20.4,0
17703,24,20.04,0
15342,12,23.4,0
14300,12,10.2,0
16146,24,40.56,0
…,…,…,…
14002,108,8.64,-1
16768,6,15.7,0
13743,36,23.4,0


In [18]:
dbs_grouped_df.plot.scatter("Quantity", "Sales", color="cluster:N")

alt.Chart(...)

### HDBSCAN 分群法


另使用 HDBSCAN 方法進行分群（不過針對這個資料的效果不佳，僅供參考）。


In [19]:
from hdbscan import HDBSCAN

clusterer = HDBSCAN(min_samples=5).fit(rfm_score_df_clustering.to_numpy())

hdbs_grouped_df = grouped_df.with_columns(cluster=pl.Series("cluster", clusterer.labels_))

In [20]:
hdbs_grouped_df

CustomerID,Quantity,Sales,cluster
i64,i64,f64,i64
16539,24,20.4,29
17703,24,20.04,29
15342,12,23.4,64
14300,12,10.2,58
16146,24,40.56,38
…,…,…,…
14002,108,8.64,-1
16768,6,15.7,80
13743,36,23.4,-1


In [21]:
hdbs_grouped_df.plot.scatter("Quantity", "Sales", color="cluster:N")

alt.Chart(...)

## 找出 RFM 分數最高的前 200 名顧客，並將其分成 3 群


### 計算 RFM 分數


找一個 Recency「今天」的基準點。


In [22]:
df["InvoiceDate"].max()

datetime.datetime(2011, 12, 9, 12, 50)

製作 R、F、M 表格。


In [23]:
from datetime import datetime

today = datetime(2011, 12, 10)  # 找一個接近目前日期的日期

rfm_df = df.group_by("CustomerID").agg(
    Recency=(today - pl.col("InvoiceDate").min()).dt.total_days(),
    Frequency=pl.col("InvoiceDate").count(),
    Monetary=pl.col("Sales").sum(),
)

In [24]:
rfm_df

CustomerID,Recency,Frequency,Monetary
i64,i64,u32,f64
14306,85,1,102.0
15092,283,3,46.8
16527,81,1,1.65
13255,9,1,14.95
17173,14,1,87.6
…,…,…,…
16643,245,1,23.8
17673,320,4,19.95
14788,7,1,11.7


RFM score: 分成 5 個級距。


In [25]:
rfm_score_df = rfm_df.with_columns(
    RecencyScore=pl.col("Recency").qcut(5, labels=["5", "4", "3", "2", "1"]),
    FrequencyScore=pl.col("Frequency").qcut(4, labels=["1", "2", "3", "4"]),
    MonetaryScore=pl.col("Monetary").qcut(5, labels=["1", "2", "3", "4", "5"]),
).with_columns(
    CustomerScore=pl.col("RecencyScore").cast(pl.Int32) * 100 + pl.col("FrequencyScore").cast(pl.Int32) * 10 + pl.col("MonetaryScore").cast(pl.Int32)
)

In [26]:
rfm_score_df

CustomerID,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,CustomerScore
i64,i64,u32,f64,cat,cat,cat,i32
14306,85,1,102.0,"""4""","""1""","""5""",415
15092,283,3,46.8,"""2""","""3""","""4""",234
16527,81,1,1.65,"""4""","""1""","""1""",411
13255,9,1,14.95,"""5""","""1""","""2""",512
17173,14,1,87.6,"""5""","""1""","""5""",515
…,…,…,…,…,…,…,…
16643,245,1,23.8,"""3""","""1""","""3""",313
17673,320,4,19.95,"""1""","""4""","""3""",143
14788,7,1,11.7,"""5""","""1""","""2""",512


拉出 RFM score 最高的前 200 名顧客。


In [27]:
# 拉出 RFM score 最高的前 200 名顧客。

top_200_customers_df = rfm_score_df.filter(
    (pl.col("RecencyScore").cast(pl.Int32) > 2) &
    (pl.col("FrequencyScore").cast(pl.Int32) > 2) &
    (pl.col("MonetaryScore").cast(pl.Int32) > 2)
).sort("CustomerScore", descending=True).head(200)

top_200_customers_df

CustomerID,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,CustomerScore
i64,i64,u32,f64,cat,cat,cat,i32
15113,54,4,117.09,"""5""","""4""","""5""",545
15527,38,4,81.54,"""5""","""4""","""5""",545
18210,51,4,95.23,"""5""","""4""","""5""",545
12536,59,6,122.66,"""5""","""4""","""5""",545
17509,63,6,90.6,"""5""","""4""","""5""",545
…,…,…,…,…,…,…,…
12360,200,6,76.66,"""3""","""4""","""5""",345
15123,206,4,108.3,"""3""","""4""","""5""",345
14132,183,6,84.6,"""3""","""4""","""5""",345


使用 KMeans 將這些客戶分成三群。


In [28]:
from typing import cast
import pandas as pd
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.set_output(transform="pandas")

scaled_df = top_200_customers_df.to_pandas().set_index("CustomerID")
transformed_scaled_df = cast(pd.DataFrame, scaler.fit_transform(scaled_df))

top_200_standardized_rfm_df = pl.from_pandas(transformed_scaled_df, include_index=True)

In [29]:
from sklearn.cluster import KMeans

rfm_score_df_clustering = top_200_standardized_rfm_df.drop("CustomerID")

clusterer = KMeans(n_clusters=3).fit(rfm_score_df_clustering)

top_200_customers_df = top_200_customers_df.with_columns(cluster=pl.Series("cluster", clusterer.labels_))

In [30]:
top_200_customers_df

CustomerID,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,CustomerScore,cluster
i64,i64,u32,f64,cat,cat,cat,i32,i32
15113,54,4,117.09,"""5""","""4""","""5""",545,2
15527,38,4,81.54,"""5""","""4""","""5""",545,2
18210,51,4,95.23,"""5""","""4""","""5""",545,2
12536,59,6,122.66,"""5""","""4""","""5""",545,2
17509,63,6,90.6,"""5""","""4""","""5""",545,2
…,…,…,…,…,…,…,…,…
12360,200,6,76.66,"""3""","""4""","""5""",345,1
15123,206,4,108.3,"""3""","""4""","""5""",345,1
14132,183,6,84.6,"""3""","""4""","""5""",345,1


In [31]:
top_200_customers_df.plot.scatter("Recency", "Frequency", size="Monetary", color="cluster:N")

alt.Chart(...)